<a href="https://colab.research.google.com/github/a-mhamdi/jlai/blob/main/Codes/Julia/Part-3/transfer_learning_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning (CIFAR'10 DATASET)
---

In [ ]:
versioninfo() # -> v"1.11.5"

In [ ]:
pkgs = """[deps]
BSON = "fbb218c0-5317-5bc6-957e-2ee96dd4b1f0"
CUDA = "052768ef-5323-5732-b1bb-66c8b64840ba"
CairoMakie = "13f3f980-e62b-5c42-98c6-ff1f3baf88f0"
DataAugmentation = "88a5189c-e7ff-4f85-ac6b-e6158070f02e"
Flux = "587475ba-b771-5e3f-ad9e-33799f191a9c"
ImageInTerminal = "d8c32880-2388-543b-8c61-d9f865259254"
ImageShow = "4e3cecfd-b093-5904-9786-8bbb286a6a31"
Images = "916415d5-f1e6-5110-898d-aaa5f9f070e0"
JLD2 = "033835bb-8acc-5ee8-8aae-3f567f8a3819"
MLDatasets = "eb30cadb-4394-5ae3-aed4-317e484a6458"
Metalhead = "dbeba491-748d-5e0e-a39e-b530a07fa0cc"
ProgressMeter = "92933f4c-e287-5a05-a399-4b506db050ca"
Statistics = "10745b16-79ce-11e8-11f9-7d13ad32a3b2"
cuDNN = "02a925ec-e4fe-4b08-9a7e-0d78e3d38ccd"
"""

open("Project.toml", "w") do file
    write(file, pkgs)
end

In [ ]:
_ = begin
  import Pkg;
  Pkg.activate(".");
  Pkg.instantiate();
end

In [ ]:
Pkg.status()

Load the pre-trained model

In [ ]:
using CUDA
using Metalhead

[API Reference](https://fluxml.ai/Metalhead.jl/dev/api/reference/#API-Reference)

In [ ]:
resnet = ResNet(18; pretrain=true).layers;

In [ ]:
using Flux
using Flux: onecold, onehotbatch

In [ ]:
mdl = Chain(
    resnet[1:end-1],
    resnet[end][1:end-1],
    # Replace the last layer
    Dense(512 => 256, relu),
    Dense(256 => 10)
) |> gpu

Load the CIFAR10 dataset

In [ ]:
using MLDatasets: CIFAR10

In [ ]:
d = CIFAR10()

In [ ]:
idx = rand(1:50000)

In [ ]:
using ImageShow

In [ ]:
using MLDatasets: convert2image

In [ ]:
convert2image(d, idx)

In [ ]:
printstyled("Label is $(d.targets[idx])"; bold=true, color=:red)

In [ ]:
function get_data(split, lm::Integer=1024)
    data = CIFAR10(split)
    X, y = data.features[:, :, :, 1:lm] ./ 255, onehotbatch(data.targets[1:lm], 0:9)
    loader = Flux.DataLoader((X, y); batchsize=16, shuffle=true) |> gpu
    return loader
end

In [ ]:
train_loader = get_data(:train, 512);
test_loader = get_data(:test, 128);

Define a setup of the optimizer

In [ ]:
loss(X, y) = Flux.Losses.logitcrossentropy(mdl(X), y)
opt = Adam(3e-3)
ps = Flux.params(mdl[3:end])

In [ ]:
for epoch in 1:8
    Flux.train!(
        loss,
        ps,
        train_loader,
        opt,
        cb = Flux.throttle(() -> println("Training"), 10)
    )
end

In [ ]:
sample = d.features[:, :, :, idx]
println(size(sample))
mb_sample = reshape(sample, 32, 32, 3, 1)
println(size(mb_sample))

In [ ]:
mdl(mb_sample) |> softmax |> onecold

In [ ]:
#=
using Optimisers
opt_state = Optimisers.setup(Adam(3e-3), mdl[3:end]) # Freeze the weights of the pre-trained layers
using ProgressMeter
epochs = 5
# Fine-tune the model
for epoch in 1:epochs
    @showprogress for (X, y) in train_loader
        # Compute the gradient of the loss with respect to the model's parameters
        ∇ = Flux.gradient( m -> loss(m, X, y), mdl)
        # Update the `mdl`'s parameters
        Flux.update!(opt_state, mdl, ∇[1])
    end
    @info "Calculate the accuracy on the test set"
    for (X, y) in test_loader
        accuracy = sum(onecold(mdll(X)) .== onecold(y)) / length(y)
        println("Epoch: $epoch, Accuracy: $accuracy")
    end
end
=#